In [73]:
import json
import csv
import os
from pprint import pprint as pr
import sqlalchemy as sa
from sqlalchemy import join
import pandas as pd
from app.sample_data import stat_vectors, comp_style_vectors, map_style_vectors, roles

In [74]:
BASE_DIR = os.path.abspath(os.path.dirname('../'))

SERVER='192.168.0.245'
SQLSERVER_DATABASE='lookingglass'
SQLSERVER_USER='thecaptain'
SQLSERVER_PASSWORD='99Redbal00ns'
PORT=1433
SQLSERVER_BASE_URI = 'mssql+pyodbc://{USER}:{PWD}@{SERVER}:{PORT}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'

engine = sa.create_engine(SQLSERVER_BASE_URI.format(
  SERVER=SERVER,
  DATABASE=SQLSERVER_DATABASE,
  USER=SQLSERVER_USER,
  PWD=SQLSERVER_PASSWORD,
  PORT=PORT
))

meta_data = sa.MetaData(bind=engine)
sa.MetaData.reflect(meta_data)

In [75]:
Player=meta_data.tables['Player']
TeamMember=meta_data.tables['TeamMember']
Hero=meta_data.tables['Hero']
Battleground=meta_data.tables['Map']
Replay=meta_data.tables['ReplayDetail']
ReplayPlayer=meta_data.tables['ReplayPlayer']

In [193]:
team_id = 4109
act_list = ['TheCaptain#12345', 'Aly#11717']

j = Replay.join(
    ReplayPlayer
    , ReplayPlayer.c.ReplayId == Replay.c.ReplayId
).join(
 Player
    , Player.c.PlayerId == ReplayPlayer.c.PlayerId
).join(
    Battleground
    , Battleground.c.MapId == Replay.c.MapId
).join(
    Hero
    , Hero.c.HeroId == ReplayPlayer.c.HeroId
)

query = sa.select(
    Replay.c.ReplayFingerPrint
    , Battleground.c.MapName
    , Replay.c.GameType
    , Replay.c.GameDate
    , Replay.c.GameLengthTimestamp
    , Player.c.Battletag
    , Hero.c.HeroName
    , Hero.c.NewRole
    , ReplayPlayer
    , Player.c.BlizzardId
).select_from(j).where(sa.and_(
    Replay.c.GameDate >= '01/01/2022'
    # , Player.c.Battletag.in_(act_list)
))

df = pd.read_sql(sql = query,
                 con = engine)

df['Battletag'].drop_duplicates(keep='first', inplace=True)
# df.Battletag = 'TheCaptain#12345'
df.shape

(46530, 57)

In [194]:
df.dtypes

ReplayFingerPrint                 object
MapName                           object
GameType                          object
GameDate                  datetime64[ns]
GameLengthTimestamp               object
Battletag                         object
HeroName                          object
NewRole                           object
PlayerReplayId                     int64
ReplayId                           int64
PlayerId                           int64
HeroId                             int64
HeroLevel                          int64
Team                               int64
IsWinner                           int64
Party                              int64
FirstToTen                         int64
Kills                              int64
Assists                            int64
Takedowns                          int64
Deaths                             int64
Level                              int64
HeroDamage                         int64
SiegeDamage                        int64
StructureDamage 

In [195]:
##Formatting the data
df['GameTypeCode'] = df.GameType.astype('category').cat.codes
df['HeroRoleCode'] = df.NewRole.astype('category').cat.codes
df['MapCode'] = df.MapName.astype('category').cat.codes
df['YearCode'] = df.GameDate.dt.year
df['MonthCode'] = df.GameDate.dt.month
df['DayCode'] = df.GameDate.dt.dayofweek
# df['BlizzardId'] = df.BlizzardId.astype('int')

##Creating a target to determine outcome
df['Target'] = (df.IsWinner == 1 ).astype('int')

df.head(5)

,ReplayFingerPrint,MapName,GameType,GameDate,GameLengthTimestamp,Battletag,HeroName,NewRole,PlayerReplayId,ReplayId,...,RegenGlobes,TimeonFire,BlizzardId,GameTypeCode,HeroRoleCode,MapCode,YearCode,MonthCode,DayCode,Target
0,e34a8cda-6f59-1fff-1ac1-f9223ac4f2af,Garden of Terror,StormLeague,2022-03-20 20:37:12,1009,TheCaptain#12345,Jaina,Ranged Assassin,595,61,...,32,225,9771889,2,3,7,2022,3,6,1
1,68c67e67-aab3-5bf3-04eb-737340b16624,Infernal Shrines,QuickMatch,2022-01-01 05:10:50,1484,TheCaptain#12345,Nazeebo,Ranged Assassin,22732,2276,...,42,118,9771889,1,3,10,2022,1,5,1
2,08186618-744c-7425-6536-34a0de482b88,Hanamura Temple,QuickMatch,2022-01-01 05:35:49,1301,TheCaptain#12345,Stukov,Healer,22736,2273,...,37,0,9771889,1,1,8,2022,1,5,1
3,588f367e-af15-7fb6-e430-c67549a14638,Blackheart's Bay,QuickMatch,2022-01-01 06:02:14,1030,TheCaptain#12345,Valla,Ranged Assassin,22749,2275,...,25,0,9771889,1,3,2,2022,1,5,0
4,80f99fd5-da0c-a1ac-e533-861e417c6063,Warhead Junction,StormLeague,2022-01-02 02:31:22,1163,TheCaptain#12345,Johanna,Tank,22762,2278,...,18,0,9771889,2,5,17,2022,1,6,0


In [196]:
df.Battletag.unique

<bound method Series.unique of 0         TheCaptain#12345
1         TheCaptain#12345
2         TheCaptain#12345
3         TheCaptain#12345
4         TheCaptain#12345
               ...        
46525    cupofjoe2236#1247
46526      ElcaRReado#1187
46527          StatiK#1917
46528        Espedual#1547
46529      Jaymaster#11674
Name: Battletag, Length: 46530, dtype: object>

In [197]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score

In [198]:
rf = RandomForestClassifier(
    n_estimators=50
    , min_samples_split=10
    , random_state=1
)

In [199]:
train = df[df.GameDate < '2022-05-01']
test = df[df.GameDate >= '2022-05-01']

In [215]:
predictors = [
    'HeroId'
    # , 'DayCode'
    , 'MapCode', 'GameTypeCode', 'HeroRoleCode'
    # , 'HeroLevel'
    # , 'Kills'
    # , 'Assists'
    # , 'Deaths'
    , 'ExperienceContribution'
    , 'FirstToTen'
    # , 'Level'
    # , 'HeroDamage'
    # , 'Healing'
    , 'Takedowns'
    # , 'SelfHealing'
    , 'DamageTaken'
    , 'TimeSpentDead'
    , 'GameLengthTimestamp'
]

In [216]:
rf.fit(train[predictors], train["Target"])

c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [217]:
preds = rf.predict(test[predictors])

c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [218]:
accuracy_score(test['Target'], preds)

0.8363453815261044

In [219]:
combined = pd.DataFrame(dict(actual=test["Target"], prediction=preds))
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,2099,391
1,424,2066


In [220]:
precision_score(test['Target'], preds)

0.8408628408628409

In [221]:
grouped_replays = df.groupby('HeroName')
group = grouped_replays.get_group('Brightwing')
group

,ReplayFingerPrint,MapName,GameType,GameDate,GameLengthTimestamp,Battletag,HeroName,NewRole,PlayerReplayId,ReplayId,...,RegenGlobes,TimeonFire,BlizzardId,GameTypeCode,HeroRoleCode,MapCode,YearCode,MonthCode,DayCode,Target
182,413218d8-6265-6879-cbe3-88f238a62497,Battlefield of Eternity,QuickMatch,2022-03-01 02:15:28,1075,TheCaptain#12345,Brightwing,Healer,28748,2875,...,14,0,9771889,1,1,1,2022,3,1,0
183,67bad1ef-b494-9c76-7ded-7954fa53b5af,Garden of Terror,QuickMatch,2022-03-01 02:36:56,1074,TheCaptain#12345,Brightwing,Healer,28761,2877,...,23,0,9771889,1,1,7,2022,3,1,1
228,9da0b59b-1a25-b9d6-2e5c-b0ace2694958,Infernal Shrines,QuickMatch,2022-03-23 23:43:22,1354,TheCaptain#12345,Brightwing,Healer,30341,3039,...,26,0,9771889,1,1,10,2022,3,2,0
362,16cfe951-a8d5-faf7-231f-439139599035,Towers of Doom,StormLeague,2022-05-08 21:10:28,1144,TheCaptain#12345,Brightwing,Healer,32450,3250,...,22,0,9771889,2,1,15,2022,5,6,1
415,f991baaf-391f-cfd7-a026-fdf8711cae58,Volskaya Foundry,QuickMatch,2022-05-22 00:09:37,916,TheCaptain#12345,Brightwing,Healer,33009,3303,...,15,0,9771889,1,1,16,2022,5,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46124,de2c91b0-0673-df55-5d6b-eedcd08b23c0,Garden of Terror,StormLeague,2022-07-01 19:33:35,1318,JUSTPEACHY#21982,Brightwing,Healer,35401,3542,...,27,0,None,2,1,7,2022,7,4,0
46163,e60d2f7a-2c03-5a0d-7646-5a441e159d45,Alterac Pass,QuickMatch,2022-07-03 04:33:20,1408,BryGuy#11187,Brightwing,Healer,35418,3548,...,33,0,None,1,1,0,2022,7,6,0
46173,87ecd0ce-b451-dd31-4584-33ac9cdef013,Hanamura Temple,StormLeague,2022-07-04 16:34:06,1186,VeganButch#1362,Brightwing,Healer,35508,3552,...,15,0,None,2,1,8,2022,7,0,0
46240,cc4f9a46-f7fe-73af-d0a8-bc726d7f9072,Hanamura Temple,StormLeague,2022-07-09 02:39:32,1049,Schmoejoe#1593,Brightwing,Healer,35714,3578,...,40,0,None,2,1,8,2022,7,5,1


In [222]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("GameDate")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [223]:
cols = [
      'Kills', 'Assists', 'Takedowns', 'Deaths'
    , 'Level', 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
    , 'GameLengthTimestamp'
]

new_cols = [f'{c}_Rolling' for c in cols]

In [224]:
rolling_averages(group, cols, new_cols)

,ReplayFingerPrint,MapName,GameType,GameDate,GameLengthTimestamp,Battletag,HeroName,NewRole,PlayerReplayId,ReplayId,...,OutnumberedDeaths_Rolling,TeamfightEscapes_Rolling,TeamfightHealing_Rolling,TeamfightDamageTaken_Rolling,TeamfightHeroDamage_Rolling,Multikill_Rolling,PhysicalDamage_Rolling,SpellDamage_Rolling,RegenGlobes_Rolling,GameLengthTimestamp_Rolling
35698,13a6f929-7e11-ba9c-0fcd-ba8ef5f9401c,Braxis Holdout,StormLeague,2022-01-01 01:15:15,935,TwiztdPnguin#1549,Brightwing,Healer,113055,11307,...,1.333333,0.0,0.0,27839.333333,9112.333333,1.333333,25196.333333,22710.000000,21.666667,1167.666667
15830,1aa5db0e-445e-dab0-7fec-f27bf28c0c6e,Dragon Shire,QuickMatch,2022-01-01 01:27:37,1395,Urayami#1552,Brightwing,Healer,110931,11097,...,0.666667,0.0,0.0,21036.666667,8398.666667,1.333333,24676.333333,24388.000000,22.666667,1084.333333
20151,507c5cb5-4703-a505-c5d7-550417e0a287,Braxis Holdout,StormLeague,2022-01-01 01:30:11,1420,UrbanPunk#11242,Brightwing,Healer,111686,11169,...,0.333333,0.0,0.0,31868.666667,10879.000000,1.000000,30869.000000,27699.333333,30.666667,1339.666667
14283,05ecd383-ff1d-ea03-4466-2dad06785339,Garden of Terror,StormLeague,2022-01-01 01:53:30,962,Jep#1749,Brightwing,Healer,111710,11173,...,0.666667,0.0,0.0,24107.000000,9824.000000,1.000000,26348.000000,22025.666667,24.333333,1250.000000
35128,6a47d35f-0e9e-bbda-b384-1eca311188c5,Sky Temple,QuickMatch,2022-01-01 02:34:08,806,soccerboy54#1695,Brightwing,Healer,110955,11099,...,0.666667,0.0,0.0,25079.666667,9967.333333,1.666667,25636.000000,24063.333333,22.333333,1259.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9823,cd68b9b1-d17b-f562-2985-74eaadbe4a19,Towers of Doom,StormLeague,2022-07-16 22:22:55,1037,JohnnyKA#1258,Brightwing,Healer,36106,3614,...,0.333333,0.0,0.0,14611.666667,5211.000000,2.000000,29663.000000,27114.000000,32.666667,1208.000000
1212,d2d80b91-a899-b2ea-fc0a-1d4cf22e82c0,Tomb of the Spider Queen,QuickMatch,2022-07-16 22:56:07,1308,BornToShine#11556,Brightwing,Healer,36078,3611,...,0.333333,0.0,0.0,19939.666667,5688.333333,1.333333,23373.333333,27449.000000,28.000000,1207.000000
1213,279042a7-9209-4142-2b08-40ded654557c,Alterac Pass,QuickMatch,2022-07-16 23:26:43,1686,BornToShine#11556,Brightwing,Healer,36074,3608,...,0.333333,0.0,0.0,22852.666667,5998.666667,2.333333,25526.333333,29567.000000,30.000000,1266.333333
687,83d5467f-21e9-5602-6769-bc303a2ab540,Tomb of the Spider Queen,StormLeague,2022-07-19 00:05:17,1306,TheCaptain#12345,Brightwing,Healer,153085,15310,...,0.000000,0.0,0.0,22215.333333,6532.000000,3.666667,34633.000000,39178.333333,31.333333,1343.666667


In [225]:
replay_rolling = df.groupby('HeroName').apply(lambda x: rolling_averages(x, cols, new_cols)).droplevel('HeroName')
replay_rolling.index = range(replay_rolling.shape[0]) # Gives unique values for each row in the index.
replay_rolling

,ReplayFingerPrint,MapName,GameType,GameDate,GameLengthTimestamp,Battletag,HeroName,NewRole,PlayerReplayId,ReplayId,...,OutnumberedDeaths_Rolling,TeamfightEscapes_Rolling,TeamfightHealing_Rolling,TeamfightDamageTaken_Rolling,TeamfightHeroDamage_Rolling,Multikill_Rolling,PhysicalDamage_Rolling,SpellDamage_Rolling,RegenGlobes_Rolling,GameLengthTimestamp_Rolling
0,21f63581-b65b-87aa-b7eb-14bc6e942264,Braxis Holdout,QuickMatch,2022-01-01 05:29:34,1268,Grimas#1740,Abathur,Support,111103,11112,...,0.333333,0.0,0.0,417.000000,57.333333,0.666667,0.000000,145801.000000,1.666667,1176.333333
1,9d10d401-1c7d-9f46-dbcb-c450114bdcbc,Blackheart's Bay,QuickMatch,2022-01-01 06:02:33,1057,Spader#11430,Abathur,Support,113301,11332,...,0.333333,0.0,0.0,417.000000,57.333333,2.000000,1833.333333,148729.000000,1.333333,1320.333333
2,0b6ee2e9-8957-cb85-0bf3-d82bd0e0d70a,Dragon Shire,QuickMatch,2022-01-01 06:24:01,1030,soakplz#1213,Abathur,Support,111184,11118,...,0.000000,0.0,0.0,0.000000,0.000000,3.000000,4218.333333,141368.666667,2.000000,1269.666667
3,e1c9b5db-fe03-0538-002e-d2be6b2f9f50,Cursed Hollow,StormLeague,2022-01-01 07:18:08,1193,Dunkerz#21913,Abathur,Support,126376,12643,...,0.333333,0.0,0.0,127.333333,0.000000,2.666667,4706.333333,114910.333333,3.000000,1118.333333
4,627b3c61-ac85-fbef-22f3-8a150ffbb812,Towers of Doom,StormLeague,2022-01-01 19:47:10,1504,ledzep154#1772,Abathur,Support,133771,13374,...,0.333333,0.0,0.0,127.333333,0.000000,3.000000,7848.000000,117247.000000,4.333333,1093.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46255,165e833f-5360-4fe3-0299-30bcd5d767ff,Garden of Terror,StormLeague,2022-07-12 00:44:24,1648,Rells#11356,Zul'jin,Ranged Assassin,35990,3595,...,2.666667,0.0,0.0,27535.000000,40955.333333,1.666667,115959.666667,105165.666667,26.333333,1422.666667
46256,59ee9da7-21e7-2f39-334e-f3e8fbf408b3,Hanamura Temple,StormLeague,2022-07-12 01:40:30,1201,Hamflaps#1789,Zul'jin,Ranged Assassin,35993,3596,...,1.333333,0.0,0.0,30780.333333,53724.666667,2.000000,132370.666667,138339.333333,33.333333,1566.666667
46257,a8a47d86-7dfb-e78b-05dd-e1af71dffc7f,Alterac Pass,StormLeague,2022-07-16 21:59:19,1201,SubZero#1577,Zul'jin,Ranged Assassin,36116,3607,...,2.000000,0.0,0.0,31963.000000,47825.000000,1.666667,106089.666667,99021.333333,29.666667,1391.000000
46258,cb460655-9cd7-05db-9ae1-609c6003f446,Garden of Terror,QuickMatch,2022-07-17 00:20:27,1349,cgsdeath#1660,Zul'jin,Ranged Assassin,36219,3621,...,2.000000,0.0,0.0,35488.333333,44132.000000,1.666667,93267.666667,81173.333333,26.000000,1350.000000


In [226]:
def make_predictions(data, predictors, rand_f):
    train = data[data.GameDate < '2022-05-01']
    test = data[data.GameDate >= '2022-05-01']
    rand_f.fit(train[predictors], train["Target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["Target"], prediction=preds), index=test.index)
    precision = precision_score(test['Target'], preds)
    return combined, precision

In [227]:
predictors = [
    'HeroId', 'MapCode', 'GameTypeCode', 'HeroRoleCode', 'GameLengthTimestamp'
    , 'HeroLevel', 'Kills', 'Assists', 'Deaths'
    , 'ExperienceContribution', 'FirstToTen', 'Level', 'HeroDamage', 'Healing'
    , 'Takedowns', 'DayCode', 'SelfHealing', 'DamageTaken', 'TimeSpentDead'
]

cols = [
      'Kills', 'Assists', 'Takedowns', 'Deaths'
    , 'Level', 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
]
new_cols = [f'{c}_Rolling' for c in cols]
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
rf.fit(train[predictors], train["Target"])
combined, precision = make_predictions(replay_rolling, predictors + new_cols, rf)
precision

c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(
c:\source\innovation\insight\lookingglass_playground\venv\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.8411251980982567

In [228]:
combined = combined.merge(replay_rolling[["GameDate", "Battletag", "HeroName", "IsWinner"]], left_index=True, right_index=True)
combined

,actual,prediction,GameDate,Battletag,HeroName,IsWinner
466,0,1,2022-05-01 02:22:03,Sammyy#11285,Abathur,0
467,1,1,2022-05-01 03:50:09,NightKnight#1170286,Abathur,1
468,1,0,2022-05-03 18:01:11,Starblade#1651,Abathur,1
469,1,1,2022-05-03 20:51:52,SavPants#1349,Abathur,1
470,1,1,2022-05-03 21:14:28,RockForge#1559,Abathur,1
...,...,...,...,...,...,...
46255,1,1,2022-07-12 00:44:24,Rells#11356,Zul'jin,1
46256,0,0,2022-07-12 01:40:30,Hamflaps#1789,Zul'jin,0
46257,0,0,2022-07-16 21:59:19,SubZero#1577,Zul'jin,0
46258,1,1,2022-07-17 00:20:27,cgsdeath#1660,Zul'jin,1


In [13]:
# df.columns
points = [
    'Battletag', 'HeroName', 'NewRole', 'GameMap'
    , 'GameDate',  'GameType', 'GameLengthTimestamp'
    , 'HeroLevel', 'Kills', 'Assists', 'Takedowns', 'Deaths', 'Team'
    , 'IsWinner', 'FirstToTen', 'Level', 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'MatchAward', 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
    , 'PlayerId','HeroId', 'ReplayId', 'SourceReplayId', 'Party', 'RankedSeason'
    # , 'LevelOne', 'LevelFour', 'LevelSeven', 'LevelTen', 'LevelThirteen', 'LevelSixteen', 'LevelTwenty'
]

agg_points = [
    'Battletag', 'Hero', 'NewRole', 'GameLength'
    , 'HeroLevel', 'Kills', 'Assists', 'Takedowns', 'Deaths', 'Team'
    , 'IsWinner', 'FirstToTen', 'Level', 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'MatchAward', 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
    # , 'GameDate',  'GameType'
    # , 'PlayerId','HeroId',
    # , 'LevelOne', 'LevelFour', 'LevelSeven', 'LevelTen', 'LevelThirteen', 'LevelSixteen', 'LevelTwenty'
]


dps_agg_points = [
    'Battletag', 'Hero', 'NewRole', 'IsWinner'
    , 'Kills', 'Assists', 'Takedowns', 'Deaths'
    , 'HighestKillStreak', 'HeroDamage'
    , 'SiegeDamage', 'StructureDamage', 'MinionDamage', 'CreepDamage'
    , 'SummonDamage', 'TimeCcEnemyHeroes', 'Healing', 'SelfHealing'
    , 'DamageTaken', 'ExperienceContribution', 'TownKills', 'TimeSpentDead'
    , 'MercCampCaptures', 'WatchTowerCaptures', 'MetaExperience'
    , 'ProtectionAllies', 'SilencingEnemies', 'RootingEnemies'
    , 'StunningEnemies', 'ClutchHeals', 'Escapes', 'Vengeance'
    , 'OutnumberedDeaths', 'TeamfightEscapes', 'TeamfightHealing'
    , 'TeamfightDamageTaken', 'TeamfightHeroDamage', 'Multikill'
    , 'PhysicalDamage', 'SpellDamage', 'RegenGlobes'
    # , 'GameDate',  'GameType'
    # , 'PlayerId','HeroId',
    # , 'LevelOne', 'LevelFour', 'LevelSeven', 'LevelTen', 'LevelThirteen', 'LevelSixteen', 'LevelTwenty'
]

In [14]:
df[[*dps_agg_points]].groupby(['Battletag', 'Hero', 'NewRole']).agg(
    MeanKills=('Kills', 'mean'), KillCount=('Kills', 'count'), MedianKills=('Kills', 'median'),
    Takedowns=('Takedowns', 'count')
)

MeanKills  KillCount  \
Battletag        Hero        NewRole                                 
TheCaptain#12345 Anub'arak   Tank              1.600000          5   
                 Auriel      Healer            0.000000          1   
                 Blaze       Tank              1.000000          1   
                 Brightwing  Healer            2.000000          8   
                 Cassia      Ranged Assassin   4.405797         69   
                 Chromie     Ranged Assassin   3.222222          9   
                 Deathwing   Bruiser           4.500000          2   
                 Deckard     Healer            0.333333          3   
                 Dehaka      Bruiser           0.000000          1   
                 Diablo      Tank              1.666667          3   
                 Fenix       Ranged Assassin   2.500000          2   
                 Gall        Ranged Assassin   1.500000          2   
                 Gazlowe     Bruiser           0.000000          1   
                 Genji       Ranged Assassin   8.000000          2   
                 Greymane    Ranged Assassin   5.000000          1   
                 Gul'dan     Ranged Assassin   3.055556         18   
                 Jaina       Ranged Assassin   4.320261        153   
                 Johanna     Tank              2.875000          8   
                 Kael'thas   Ranged Assassin   4.541667         24   
                 Kerrigan    Melee Assassin    4.000000          1   
                 Kharazim    Healer            3.333333          3   
                 Li-Ming     Ranged Assassin   4.000000         35   
                 Lt. Morales Healer            1.000000          3   
                 Mal'Ganis   Tank              4.000000          1   
                 Malfurion   Healer            1.666667          3   
                 Mephisto    Ranged Assassin   1.900000         10   
                 Nazeebo     Ranged Assassin   3.000000         14   
                 Orphea      Ranged Assassin   4.543478         46   
                 Probius     Ranged Assassin   2.000000         30   
                 Qhira       Melee Assassin    3.900000         10   
                 Raynor      Ranged Assassin   4.000000          1   
                 Rehgar      Healer            0.500000          2   
                 Sgt. Hammer Ranged Assassin   2.750000          4   
                 Stitches    Tank              2.333333          3   
                 Stukov      Healer            3.257143         35   
                 Sylvanas    Ranged Assassin   2.000000          2   
                 The Butcher Melee Assassin    3.000000          2   
                 Thrall      Bruiser           4.000000          5   
                 Tyrande     Healer            0.500000          2   
                 Uther       Healer            2.000000          2   
                 Valeera     Melee Assassin    5.800000          5   
                 Valla       Ranged Assassin   4.594595         37   
                 Varian      Bruiser           4.310811         74   
                 Whitemane   Healer            1.000000          2   
                 Zagara      Ranged Assassin   5.000000          1   
                 Zarya       Support           3.750000          8   
                 Zul'jin     Ranged Assassin   4.000000          2   

                                              MedianKills  Takedowns  
Battletag        Hero        NewRole                                  
TheCaptain#12345 Anub'arak   Tank                     1.0          5  
                 Auriel      Healer                   0.0          1  
                 Blaze       Tank                     1.0          1  
                 Brightwing  Healer                   1.0          8  
                 Cassia      Ranged Assassin          4.0         69  
                 Chromie     Ranged Assassin          1.0          9  
                 Deathwing   Bruiser           

In [224]:
df[[*dps_agg_points]].groupby(['Battletag', 'Hero', 'NewRole']).agg([ "max", "sum", "count", "mean"], axis=0)

IsWinner                     Kills  \
                                            max sum count      mean   max   
Battletag    Hero      NewRole                                              
Aly#11717    Anub'arak Tank                   1  10    12  0.833333     3   
             Cassia    Ranged Assassin        1  26    44  0.590909    10   
             Dehaka    Bruiser                0   0     4  0.000000     1   
             Diablo    Tank                   0   0     2  0.000000     3   
             Fenix     Ranged Assassin        0   0     2  0.000000     0   
...                                         ...  ..   ...       ...   ...   
YipYip#11141 Imperius  Bruiser                0   0     2  0.000000     0   
             Lunara    Ranged Assassin        1   8    16  0.500000     9   
             Raynor    Ranged Assassin        1  12    16  0.750000     4   
             Sylvanas  Ranged Assassin        1   4    14  0.285714     7   
             Valla     Ranged Assassin        1   6    14  0.428571    15   

                                                            Assists       ...  \
                                        sum count      mean     max  sum  ...   
Battletag    Hero      NewRole                                            ...   
Aly#11717    Anub'arak Tank              20    12  1.666667      13  100  ...   
             Cassia    Ranged Assassin  156    44  3.545455      23  408  ...   
             Dehaka    Bruiser            2     4  0.500000      11   30  ...   
             Diablo    Tank               6     2  3.000000      11   22  ...   
             Fenix     Ranged Assassin    0     2  0.000000       3    6  ...   
...                                     ...   ...       ...     ...  ...  ...   
YipYip#11141 Imperius  Bruiser            0     2  0.000000      11   22  ...   
             Lunara    Ranged Assassin   84    16  5.250000      24  162  ...   
             Raynor    Ranged Assassin   34    16  2.125000      17  124  ...   
             Sylvanas  Ranged Assassin   68    14  4.857143      18  160  ...   
             Valla     Ranged Assassin   80    14  5.714286      12  104  ...   

                                       PhysicalDamage                 \
                                                count           mean   
Battletag    Hero      NewRole                                         
Aly#11717    Anub'arak Tank                        12   19936.333333   
             Cassia    Ranged Assassin             44   32339.227273   
             Dehaka    Bruiser                      4   22302.000000   
             Diablo    Tank                         2   27723.000000   
             Fenix     Ranged Assassin              2   56330.000000   
...                                               ...            ...   
YipYip#11141 Imperius  Bruiser                      2   34818.000000   
             Lunara    Ranged Assassin             16   50279.500000   
             Raynor    Ranged Assassin             16  105575.750000   
             Sylvanas  Ranged Assassin             14   76491.428571   
             Valla     Ranged Assassin             14   73247.142857   

                                       SpellDamage                 \
                                               max      sum count   
Battletag    Hero      NewRole                                      
Aly#11717    Anub'arak Tank                  90282   654482    12   
             Cassia    Ranged Assassin      195268  4506456    44   
             Dehaka    Bruiser               66791   264456     4   
             Diablo    Tank                  81278   162556     2   
             Fenix     Ranged Assassin       34956    69912     2   
...                                            ...      ...   ...   
YipYip#11141 Imperius  Bruiser               47113    94226     2   
             Lunara    Ranged Assassin      136293  1606160    16   
             Raynor    Ranged Assassin       45022   3925

In [225]:
dfn = df[[*dps_agg_points]].groupby(['Battletag', 'Hero', 'NewRole']).agg(
    HeroCount=('Kills', 'count')
    , Wins=('IsWinner', 'sum')
).sort_values('HeroCount', ascending=False)

dfn['WinRate'] = round(dfn.Wins / dfn.HeroCount, 2)

dfn.groupby(['Battletag']).head(5).sort_index()
# df.sort_values('Count', ascending=False).groupby('Date').head(2).sort_index()

HeroCount  Wins  WinRate
Battletag         Hero       NewRole                                  
Aly#11717         Cassia     Ranged Assassin         44    26     0.59
                  Jaina      Ranged Assassin         80    52     0.65
                  Orphea     Ranged Assassin         86    48     0.56
                  Valla      Ranged Assassin         22     4     0.18
                  Varian     Bruiser                 50    26     0.52
Anaressa#1611     Johanna    Tank                    52    24     0.46
                  Lunara     Ranged Assassin        116    84     0.72
                  Raynor     Ranged Assassin         52    30     0.58
                  Sylvanas   Ranged Assassin        124    64     0.52
                  Valla      Ranged Assassin         80    42     0.52
BornToShine#11556 Anduin     Healer                 218   116     0.53
                  Brightwing Healer                 180    84     0.47
                  Deckard    Healer                  40    24     0.60
                  Rehgar     Healer                 154    86     0.56
                  Stukov     Healer                 224   122     0.54
DirePants#125315  Anub'arak  Tank                    86    48     0.56
                  Dehaka     Bruiser                 40    22     0.55
                  E.T.C.     Tank                    56    26     0.46
                  Johanna    Tank                    64    44     0.69
                  Leoric     Bruiser                 42    32     0.76
Direpants#125315  Diablo     Tank                     2     2     1.00
                  Malthael   Bruiser                  2     2     1.00
Knivers#1675      Anub'arak  Tank                    42    24     0.57
                  Blaze      Tank                    30    18     0.60
                  E.T.C.     Tank                    30    18     0.60
                  Garrosh    Tank                    48    28     0.58
                  Muradin    Tank                    34    18     0.53
MowBWan#1362      Anub'arak  Tank                    69    35     0.51
                  Diablo     Tank                    72    29     0.40
                  Johanna    Tank                   169    87     0.51
                  Muradin    Tank                   156    89     0.57
                  Sonya      Bruiser                 31    20     0.65
PapaShango#11954  Artanis    Bruiser                  2     0     0.00
SubZero#1577      Greymane   Ranged Assassin        104    64     0.62
                  Sonya      Bruiser                 32    22     0.69
                  Tassadar   Ranged Assassin         58    22     0.38
                  Tychus     Ranged Assassin         90    54     0.60
                  Zul'jin    Ranged Assassin         84    46     0.55
TheCaptain#12345  Cassia     Ranged Assassin        112    72     0.64
                  Jaina      Ranged Assassin        272   156     0.57
                  Orphea     Ranged Assassin         90    46     0.51
                  Valla      Ranged Assassin         72    32     0.44
                  Varian     Bruiser                 98    44     0.45
YipYip#11141      Cassia     Ranged Assassin         10     6     0.60
                  Lunara     Ranged Assassin         16     8     0.50
                  Raynor     Ranged Assassin         16    12     0.75
                  Sylvanas   Ranged Assassin         14     4     0.29
                  Valla      Ranged Assassin         14     6     0.43